In [ ]:
# Creating a virtual environment for using hugging face model

!python -m venv hf_env



In [2]:
# Activate the virtual environment

# On macOS/Linux:
!source hf_env/bin/activate

In [ ]:
pip install transformers # Hugging face model
pip install torch       # PyTorch model 


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import pipeline       # HF API for zero shot classification
# zero shot classification for classifying text into categories labels without training the model on example for specific labels

# defining the broad categories for classification
fiction_categories = ["Fiction", "NonFiction"]

# Loading the zero shot classifier with BART model on Apple Silicon GPU
pipe = pipeline("zero-shot-classification",
                model = "facebook/bart-large-mnli",
                device = "mps")


Device set to use mps


In [ ]:
# panda to load dataset
import pandas as pd

# reading the .csv file
books = pd.read_csv('books_cleaned.csv')

In [ ]:
# saving the tagged data to a .txt file

books['tagged_description'].to_csv('tagged_description.txt', 
                                   sep = '\n', 
                                   index = False, 
                                   header = False)

In [ ]:
# FEtching first book description for Fiction category

books.loc[books["categories"] == "Fiction", "description"].reset_index(drop = 'true')[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [8]:
sequence = books.loc[books["categories"] == "Fiction", "description"].reset_index(drop = 'true')[0]

In [ ]:
# runnig the zero shot classifier

pipe(sequence, fiction_categories)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [ ]:
import numpy as np

# manually finding the highest score for predictions
max_index = np.argmax(pipe(sequence, fiction_categories)["scores"])
max_label = pipe( sequence, fiction_categories)["labels"][max_index]
max_label

'Fiction'

In [ ]:
# creating a function to predict & fetch top value of label prediction

def generate_predictions(sequence, categories):

    predictions = pipe(sequence, categories)

    max_index = np.argmax(predictions["scores"])
    max_label = predictions["labels"][max_index]

    return max_label

In [ ]:
from tqdm import tqdm

# store the actual and predcted labels
actual_categories = []
predicted_categories = []

# iterating through the 300 books samples of Fiction category

for i in tqdm(range(0, 300)):

    sequence = books.loc[books["categories"] == "Fiction", "description"].reset_index(drop = True)[i]

    predicted_categories += [generate_predictions(sequence, fiction_categories)]

    actual_categories += ["Fiction"]

100%|██████████| 300/300 [15:12<00:00,  3.04s/it]


In [ ]:
# Fetching the unique categories

print(books["categories"].unique())

['Fiction' 'Detective and mystery stories' 'Christian life' 'Africa, East'
 'Hyland, Morn (Fictitious character)' 'Adventure stories'
 'Arthurian romances' 'Fantasy fiction' nan 'English drama' 'Country life'
 'English fiction' 'Clergy' 'Aubrey, Jack (Fictitious character)'
 'Detective and mystery stories, English' 'Black Death' 'Human cloning'
 'Science fiction' 'Great Britain' 'American fiction' 'American essays'
 'Capitalism' 'Ireland' 'Juvenile Fiction' 'Literary Collections'
 'Beresford, Tommy (Fictitious character)' 'Imaginary wars and battles'
 'Dysfunctional families' 'Poirot, Hercule (Fictitious character)'
 'Christmas stories' 'Marple, Jane (Fictitious character)' 'Belgians'
 'Baggins, Frodo (Fictitious character)' 'Cambridge (Mass.)'
 'Business enterprises' 'Emotional problems'
 'Characters and characteristics in motion pictures'
 'Fantasy fiction, English' 'Fairy tales, English' 'Parenthood' 'FICTION'
 'Biography & Autobiography' 'Vietnam War, 1961-1975'
 'Computer programm

In [ ]:
# Finding all nonFiction description

nonfiction_descriptions = books.loc[books["categories"] == "NonFiction", "description"].reset_index(drop=True)

# loop through nonFiction
for i in tqdm(range(len(nonfiction_descriptions))):
    sequence = nonfiction_descriptions[i]
    predicted_categories += [generate_predictions(sequence, fiction_categories)]
    actual_categories += ["NonFiction"]


0it [00:00, ?it/s]


In [15]:
for i in tqdm(range(0, 300)):
    # Filter the data for "NonFiction" category and reset the index
    description_series = books.loc[books["categories"] == "NonFiction", "description"].reset_index(drop=True)
    
    # Ensure the series has enough items to index into
    if i < len(description_series):
        sequence = description_series.iloc[i]  # Use iloc for safe positional indexing
        
        # Get predictions
        predicted_categories += [generate_predictions(sequence, fiction_categories)]
        actual_categories += ["NonFiction"]
    else:
        # Handle the case where the index is out of bounds
        print(f"Warning: Index {i} out of range for NonFiction category.")
        break  # or continue, depending on how you want to handle it


  0%|          | 0/300 [00:00<?, ?it/s]

In [16]:
for i in tqdm(range( 0, 300)):

    sequence = books.loc[books["categories"] == "NonFiction", "description"].reset_index(drop = True)[i]

    predicted_categories += [generate_predictions(sequence, fiction_categories)]

    actual_categories += ["NonFiction"]

  0%|          | 0/300 [00:00<?, ?it/s]


KeyError: 0

In [ ]:
# Evaluating accuracy

predictions_df = pd.DataFrame({"actual_categories" : actual_categories, "predicted_categories" : predicted_categories})


In [18]:
predictions_df

,actual_categories,predicted_categories
0,Fiction,Fiction
1,Fiction,NonFiction
2,Fiction,NonFiction
3,Fiction,NonFiction
4,Fiction,NonFiction
...,...,...
295,Fiction,Fiction
296,Fiction,NonFiction
297,Fiction,NonFiction
298,Fiction,NonFiction


In [ ]:
# adding a column : if prediction is correct 

predictions_df["correct_prediction"] = (

    np.where(predictions_df["actual_categories"] == predictions_df["predicted_categories"], 1, 0)
)

In [ ]:
# Fetcing the accuracy of the zero shot prediction

predictions_df["correct_prediction"].sum() / len(predictions_df)

np.float64(0.4633333333333333)

In [ ]:
isbns = []
predicted_categories = []

# Finding the values for books with missing categories
missing_categories = books.loc[books["categories"].isna(), ["isbn13", "description"]].reset_index(drop = True)

In [ ]:
# Predicting missing categories

for i in tqdm(range(0, len(missing_categories))):

    sequence = missing_categories["description"][i]

    predicted_categories += [generate_predictions(sequence, fiction_categories)]

    isbns += [missing_categories["isbn13"][i]]


100%|██████████| 30/30 [00:45<00:00,  1.53s/it]


In [ ]:
# a dataframe with prediction for missing rows

missing_predicted_df = pd.DataFrame({"isbn13": isbns, "predicted_categories" : predicted_categories})

In [ ]:
missing_predicted_df

,isbn13,predicted_categories
0,9780006486145,NonFiction
1,9780099389019,Fiction
2,9780141311883,Fiction
3,9780141318301,NonFiction
4,9780192823311,Fiction
5,9780261102255,NonFiction
6,9780330257312,Fiction
7,9780345484291,Fiction
8,9780345915597,Fiction
9,9780349102504,Fiction


In [ ]:
# left join for book dataframe to merge predictions 
books = pd.merge(books, missing_predicted_df, on = "isbn13", how="left")

In [ ]:
# Filling the missing categories with predictions
books["categories"] = np.where(books["categories"].isna(), books["predicted_categories"], books["categories"])

# dropping the column as not needed further after merging the predicted values with the mreal existing values
books = books.drop(columns= ["predicted_categories"])

In [26]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web:A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that:Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [ ]:
# Filtering certain genres

books[books["categories"].str.lower().isin([
    "romance",
    "science fiction",
    "scifi",
    "fantasy",
    "horror",
    "mystery",
    "thriller",
    "comedy",
    "crime",
    "historical fiction"
])]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
24,9780006513087,0006513085,Gravity,Tess Gerritsen,Science fiction,http://books.google.com/books/content?id=KI66c...,Emma Watson a research physician has been trai...,2004.0,4.04,342.0,8024.0,Gravity,9780006513087 Emma Watson a research physician...
475,9780099410355,0099410354,Traitor,Matthew Woodring Stover,Science fiction,http://books.google.com/books/content?id=VbICO...,"From the depths of catastrophe, a glimmer of h...",2002.0,4.00,320.0,6765.0,Traitor,"9780099410355 From the depths of catastrophe, ..."
478,9780099422341,0099422344,Yeats is Dead!,Joseph O'Connor,Comedy,http://books.google.com/books/content?id=DrE3I...,"In aid of Amnesty International, this is a bri...",2002.0,3.39,298.0,34.0,Yeats is Dead!:A Novel by Fifteen Irish Writers,"9780099422341 In aid of Amnesty International,..."
491,9780099446729,0099446723,Blackwood Farm,Anne Rice,Horror,http://books.google.com/books/content?id=cIn8T...,"Lestat Is Back, Saviour And Demon, Presiding O...",2003.0,3.86,774.0,26145.0,Blackwood Farm,"9780099446729 Lestat Is Back, Saviour And Demo..."
542,9780099911708,0099911701,Cross Stitch,Diana Gabaldon,Historical fiction,http://books.google.com/books/content?id=0Pm4i...,"In 1945, Claire Randall is back from the war a...",1992.0,4.22,864.0,2319.0,Cross Stitch,"9780099911708 In 1945, Claire Randall is back ..."
1090,9780261102422,0261102427,The Silmarillion,John Ronald Reuel Tolkien,Fantasy,http://books.google.com/books/content?id=22ePu...,Tolkien's Silmarillion is the core work of the...,1999.0,3.91,384.0,253.0,The Silmarillion,9780261102422 Tolkien's Silmarillion is the co...
1435,9780340837955,0340837950,Stranger in a Strange Land,Robert A. Heinlein,Science fiction,http://books.google.com/books/content?id=ZQhiP...,"Epic, entertaining, Stranger in a Strange Land...",2005.0,3.92,672.0,563.0,Stranger in a Strange Land,"9780340837955 Epic, entertaining, Stranger in ..."
1439,9780345251220,0345251229,Visions from Nowhere,William Arrow,Science fiction,NaN,"The first novel in the series, ""Return to the ...",1976.0,3.23,183.0,10.0,Visions from Nowhere,"9780345251220 The first novel in the series, ""..."
2845,9780575075597,0575075597,Replay,Ken Grimwood,Fantasy,http://books.google.com/books/content?id=9vmNP...,At forty-three Jeff Winston is tired of his lo...,2005.0,4.16,272.0,412.0,Replay,9780575075597 At forty-three Jeff Winston is t...
2860,9780590254762,0590254766,"The lion, the witch and the wardrobe",Clive Staples Lewis,Fantasy,NaN,Four English school children enter the magic l...,1995.0,4.21,189.0,860.0,"The lion, the witch and the wardrobe",9780590254762 Four English school children ent...


In [28]:
# saving dataset to a file

books.to_csv("books_categories.csv", index = False)